#**Sistemi Digitali M 2020/21 - UNIBO** 
##by Pier Domenico d'Amicis

In [ ]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


For the code cell below you should see ‘Found GPU’ and tf version 1.x.
Remember to change runtime to GPU

In [ ]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
   raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))
print(tf.__version__)

Found GPU at: /device:GPU:0
1.15.2


Clone the Tensorflow API repo

In [ ]:
!git clone https://github.com/tensorflow/models.git

Cloning into 'models'...
remote: Enumerating objects: 870, done.
remote: Counting objects: 100% (870/870), done.
remote: Compressing objects: 100% (540/540), done.
remote: Total 54862 (delta 412), reused 721 (delta 315), pack-reused 53992
Receiving objects: 100% (54862/54862), 571.32 MiB | 26.62 MiB/s, done.
Resolving deltas: 100% (37496/37496), done.


Install some needed tools and dependencies.

In [ ]:
%cd "/content/models/research"
!mv object_detection/ /content/
!mv slim/ /content/
%cd "/content"
%rm -rf models

/content/models/research
/content


In [ ]:
!apt-get install protobuf-compiler python-pil python-lxml python-tk
!pip install Cython

Reading package lists... Done
Building dependency tree       
Reading state information... Done
protobuf-compiler is already the newest version (3.0.0-9.1ubuntu1).
python-tk is already the newest version (2.7.17-1~18.04).
The following additional packages will be installed:
  python-bs4 python-chardet python-html5lib python-olefile
  python-pkg-resources python-six python-webencodings
Suggested packages:
  python-genshi python-lxml-dbg python-lxml-doc python-pil-doc python-pil-dbg
  python-setuptools
The following NEW packages will be installed:
  python-bs4 python-chardet python-html5lib python-lxml python-olefile
  python-pil python-pkg-resources python-six python-webencodings
0 upgraded, 9 newly installed, 0 to remove and 30 not upgraded.
Need to get 1,615 kB of archives.
After this operation, 8,907 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 python-bs4 all 4.6.0-1 [67.9 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/main amd

In [ ]:
%cd /content/
!protoc object_detection/protos/*.proto --python_out=.

/content


In [ ]:
import os
os.environ['PYTHONPATH'] += ':/content/:/content/slim'
%cd /content/slim

/content/slim


Always run the cell below for every session restart

In [ ]:
!python setup.py build
!python setup.py install

Run Code below to check if all we need for the training has been installed.

In [ ]:
%cd /content/object_detection/builders/
!python ./model_builder_tf1_test.py

Clone my own repo to get

*   a python script to converter dataset to tfrecord
*   labels.pbtxt
*   mobilnet_ssd_v1 modified config file (network.config) 


In [ ]:
%cd /content/object_detection/
%mkdir training
%rm -rf Sistemi_Digitali_M
!git clone https://github.com/P-damicis/Sistemi_Digitali_M.git
!mv Sistemi_Digitali_M/generate_tfrecord.py ./
!mv Sistemi_Digitali_M/network.config training/
!mv Sistemi_Digitali_M/labels.pbtxt training/
%rm -rf Sistemi_Digitali_M

Import my own dataset from roboflow.com

In [ ]:
%cd /content/object_detection
%mkdir Images
%cd Images
!curl -L "https://app.roboflow.com/ds/rhLbyxam2T?key=cap0YQj8DX" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip
%rm -rf valid
!mv train/_annotations.csv ../data/train_labels.csv
!mv test/_annotations.csv ../data/test_labels.csv


###Conversion to obtain **tfrecord**

In [ ]:
%cd /content/object_detection/

# Create train data:
!python generate_tfrecord.py \
--label='Medicine-box' \
--csv_input=data/train_labels.csv \
--img_path=Images/train \
--output_path=data/train.record

# Create test data:
!python generate_tfrecord.py \
--label='Medicine-box' \
--csv_input=data/test_labels.csv \
--img_path=Images/test \
--output_path=data/test.record


Note: train.py is in legacy folder

#**TRAINING**

In [ ]:
%cd /content/object_detection/legacy/
!python train.py \
--logtostderr \
--train_dir=../training/checkpoints/ \
--pipeline_config_path=../training/network.config

Output streaming troncato alle ultime 5000 righe.
INFO:tensorflow:global step 558: loss = 5.5274 (0.480 sec/step)
I0330 12:10:45.341662 140417495144320 learning.py:512] global step 558: loss = 5.5274 (0.480 sec/step)
INFO:tensorflow:global step 559: loss = 5.7640 (0.479 sec/step)
I0330 12:10:45.822224 140417495144320 learning.py:512] global step 559: loss = 5.7640 (0.479 sec/step)
INFO:tensorflow:global step 560: loss = 5.3090 (0.482 sec/step)
I0330 12:10:46.305692 140417495144320 learning.py:512] global step 560: loss = 5.3090 (0.482 sec/step)
INFO:tensorflow:global step 561: loss = 5.1739 (0.482 sec/step)
I0330 12:10:46.789488 140417495144320 learning.py:512] global step 561: loss = 5.1739 (0.482 sec/step)
INFO:tensorflow:global step 562: loss = 5.4598 (0.482 sec/step)
I0330 12:10:47.272742 140417495144320 learning.py:512] global step 562: loss = 5.4598 (0.482 sec/step)
INFO:tensorflow:global step 563: loss = 4.7447 (0.499 sec/step)
I0330 12:10:47.773242 140417495144320 learning.py:5

##Exporting inference graph from trained checkpoint 

In [ ]:
%cd /content/object_detection/training/checkpoints/
%ls

Save inference graph


In [ ]:
%cd /content/object_detection/

!python export_inference_graph.py \
--input_type image_tensor \
--pipeline_config_path training/network.config \
--trained_checkpoint_prefix training/checkpoints/model.ckpt-15000 \
--output_directory trained_inference_graph/

###Preparing to TFLite Conversion

In [ ]:
import tensorflow as tf

In [ ]:
%cd /content/object_detection

!python export_tflite_ssd_graph.py \
--pipeline_config_path training/network.config \
--trained_checkpoint_prefix trained_inference_graph/model.ckpt \
--output_directory /content/tflite \
--add_postprocessing_op false \
--max_detections 1 \


#**Conversion to TFlite Model**

In [ ]:
!pip install tensorflow==2.1.0
!pip install tf-nightly
import tensorflow as tf

if we want to quantize int8 we have to create the function *representative_dataset* that is required by the converter.

This function provides a sample of images taken from the train set

In [ ]:
import numpy as np
import tensorflow as tf
import os
import matplotlib.pyplot as plt

def representative_dataset():
  imagespath = "/content/object_detection/Images/train/"
  for file in os.listdir(imagespath):
    data = plt.imread(imagespath+file)
    data = np.expand_dims(data, axis=0)
    yield [data.astype(np.float32)]

tf.__version__

Creating TFLite converter to convert our frozen graph.

In [ ]:
import tensorflow as tf
localpb = '/content/tflite/tflite_graph.pb'
tflite_file = '/content/mobilenet_int8.tflite'
converter = tf.compat.v1.lite.TFLiteConverter.from_frozen_graph(
  localpb, 
  ['normalized_input_image_tensor'], 
  ['TFLite_Detection_PostProcess'],
  input_shapes={"normalized_input_image_tensor" : [1, 416, 416, 3]}
)

converter.experimental_new_converter = True
converter.allow_custom_ops = True

######  NO QUANTIZATION ######
#converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS,
#                                       tf.lite.OpsSet.SELECT_TF_OPS]


######  INT 8    ######
#converter.target_spec.supported_ops = [tf.lite.OpsSet.SELECT_TF_OPS,
#                                       tf.lite.OpsSet.TFLITE_BUILTINS_INT8]

converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
converter.representative_dataset = representative_dataset

#converter.inference_input_type = tf.uint8
#converter.inference_output_type = tf.uint8

tflite_model = converter.convert()
open(tflite_file,'wb').write(tflite_model)